In [ ]:
!pip install boto3
!pip install pandas

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.9/139.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 55.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.2/85.2 kB 4.7 MB/s eta 0:00:00


In [ ]:
import pandas as pd

# Step 1: Create the data
data = {
    'ipl_team': ['CSK', 'MI', 'RCB', 'SRH', 'KKR'],
    'ipl_wins': [4, 5, 0, 2, 3],
    'ipl_team_score': [2850, 2900, 2700, 2750, 2800]
}

# Step 2: Create a DataFrame
df = pd.DataFrame(data)

# Step 3: Save to CSV
filename = 'ipl_teams.csv'
df.to_csv(filename, index=False)

# Step 4: Download CSV in Google Colab
from google.colab import files
files.download(filename)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>


# Python Code Explanation for AWS Lambda

This document provides a line-by-line explanation of the provided Python code, formatted for easy viewing and understanding in a Google Colab notebook.

### `import json, boto3, pandas as pd, os`

```
import json, boto3, pandas as pd, os

```

-   **Explanation:** This line imports the necessary Python libraries.
    
    -   `json`: For handling JSON (JavaScript Object Notation) data, though not directly used for parsing in this specific snippet, it's often included in Lambda functions.
        
    -   `boto3`: The Amazon Web Services (AWS) SDK for Python, used to interact with AWS services like S3.
        
    -   `pandas as pd`: A powerful data manipulation and analysis library, aliased as `pd` for convenience.
        
    -   `os`: Provides a way of using operating system dependent functionality, specifically for accessing environment variables here.
        

### `s3_client = boto3.client('s3')`

```
s3_client = boto3.client('s3')

```

-   **Explanation:** Initializes a client object for Amazon S3 (Simple Storage Service). This client will be used to perform operations like getting and putting objects in S3 buckets.
    

### `bucket_name = os.environ.get('S3_BUCKET_NAME')`

```
bucket_name = os.environ.get('S3_BUCKET_NAME')

```

-   **Explanation:** Retrieves the name of the S3 bucket from the Lambda function's environment variables. Environment variables are a common way to configure Lambda functions without hardcoding values in the code.
    

### `input_file_key = 'ipl_data.csv'`

```
input_file_key = 'ipl_data.csv'

```

-   **Explanation:** Defines the key (path/name) of the input CSV file within the S3 bucket. This is the file that the Lambda function will download and process.
    

### `output_file_key = 'processed_ipl_data.csv'`

```
output_file_key = 'processed_ipl_data.csv'

```

-   **Explanation:** Defines the key (path/name) for the output processed CSV file. After processing, the modified data will be uploaded back to S3 with this name.
    

### `response = s3_client.get_object(Bucket=bucket_name, Key=input_file_key)`

```
response = s3_client.get_object(Bucket=bucket_name, Key=input_file_key)

```

-   **Explanation:** This line makes an API call to S3 to retrieve the specified input file (`ipl_data.csv`) from the `bucket_name`. The result of this operation (including the file's content) is stored in the `response` variable.
    

### `csv_content = response['Body'].read().decode('utf-8')`

```
csv_content = response['Body'].read().decode('utf-8')

```

-   **Explanation:**
    
    -   `response['Body']`: Accesses the `Body` stream from the S3 `get_object` response, which contains the file's content.
        
    -   `.read()`: Reads the entire content of the stream into bytes.
        
    -   `.decode('utf-8')`: Decodes these bytes into a UTF-8 encoded string, which is necessary for `pandas` to read it as text.
        

### `df = pd.read_csv(pd.io.common.StringIO(csv_content))`

```
df = pd.read_csv(pd.io.common.StringIO(csv_content))

```

-   **Explanation:**
    
    -   `pd.io.common.StringIO(csv_content)`: This is a crucial step. `StringIO` is a module that allows you to treat a string as if it were a file. Pandas' `read_csv` function typically expects a file path or a file-like object. By wrapping `csv_content` (which is a string) in `StringIO`, we make it appear as a file to `read_csv`.
        
    -   `pd.read_csv(...)`: Reads the CSV data from the `StringIO` object directly into a pandas DataFrame, named `df`.
        

### `df['can_win'] = ((df['ipl_wins'] >= 3) & (df['ipl_team_score'] >= 2800)).astype(int)`

```
df['can_win'] = ((df['ipl_wins'] >= 3) & (df['ipl_team_score'] >= 2800)).astype(int)

```

-   **Explanation:** This is the core data preprocessing logic.
    
    -   `df['ipl_wins'] >= 3`: Creates a boolean Series where `True` indicates teams with 3 or more IPL wins.
        
    -   `df['ipl_team_score'] >= 2800`: Creates another boolean Series where `True` indicates teams with an IPL team score of 2800 or more.
        
    -   `&`: This is the bitwise AND operator, used here for element-wise logical AND between the two boolean Series. A team `can_win` only if _both_ conditions are `True`.
        
    -   `.astype(int)`: Converts the resulting boolean Series (`True`/`False`) into integers (`1`/`0`). `1` represents `True` (the team can win), and `0` represents `False` (the team cannot win).
        
    -   `df['can_win'] = ...`: Assigns this new Series as a new column named `can_win` to the DataFrame `df`.
        

### `processed_csv_content = df.to_csv(index=False)`

```
processed_csv_content = df.to_csv(index=False)

```

-   **Explanation:**
    
    -   `df.to_csv()`: Converts the modified DataFrame `df` back into a CSV formatted string.
        
    -   `index=False`: This argument is important. By default, `to_csv` writes the DataFrame's index as the first column in the CSV. Setting `index=False` prevents this, resulting in a cleaner CSV output.
        

### `s3_client.put_object(Bucket=bucket_name, Key=output_file_key, Body=processed_csv_content)`

```
s3_client.put_object(Bucket=bucket_name, Key=output_file_key, Body=processed_csv_content)

```

-   **Explanation:** This line uploads the `processed_csv_content` (the string containing the processed CSV data) back to the specified S3 `bucket_name` with the `output_file_key` (`processed_ipl_data.csv`).
    

### `return {'statusCode': 200, ...}`

```
return {'statusCode': 200, 'body': json.dumps('CSV processed and uploaded successfully!')}

```

-   **Explanation:** This is the standard return format for an AWS Lambda function.
    
    -   `'statusCode': 200`: Indicates a successful execution (HTTP 200 OK).
        
    -   `'body': json.dumps(...)`: The response body, which is typically a JSON string. Here, it's a success message. `json.dumps()` converts a Python dictionary/string into a JSON formatted string.
        

### `except Exception as e: ...`

```
except Exception as e:
    print(f"Error processing file: {e}")
    return {'statusCode': 500, 'body': json.dumps(f"Error: {e}")}

```

-   **Explanation:** This is a basic error handling block.
    
    -   `except Exception as e:`: Catches any general exception that might occur during the execution of the `try` block.
        
    -   `print(f"Error processing file: {e}")`: Prints the error message to the Lambda logs (CloudWatch Logs), which is crucial for debugging.
        
    -   `return {'statusCode': 500, ...}`: Returns an error response to the caller, indicating a server error (HTTP 500 Internal Server Error) along with the specific error message.

In [ ]:
import json
import boto3
import pandas as pd
import os

# Initialize S3 client
s3_client = boto3.client('s3')

def lambda_handler(event, context):
    # Line 1: Get the S3 bucket name from environment variables
    bucket_name = os.environ.get('S3_BUCKET_NAME')
    # Line 2: Define the input file key (path) in S3
    input_file_key = 'ipl_data.csv'
    # Line 3: Define the output file key (path) in S3 for processed data
    output_file_key = 'processed_ipl_data.csv'

    print(f"Reading {input_file_key} from bucket {bucket_name}")

    try:
        # Line 4: Download the CSV file from S3
        response = s3_client.get_object(Bucket=bucket_name, Key=input_file_key)
        # Line 5: Read the content of the file
        csv_content = response['Body'].read().decode('utf-8')
        # Line 6: Use pandas to read the CSV content into a DataFrame
        df = pd.read_csv(pd.io.common.StringIO(csv_content))

        print("Original DataFrame:")
        print(df.head())

        # Line 7: Simple preprocessing: Create a 'target' column (binary classifier)
        # Predict if a team can win based on ipl_wins and ipl_team_score
        # For simplicity, let's say a team "can win" if ipl_wins >= 3 AND ipl_team_score >= 2800
        df['can_win'] = ((df['ipl_wins'] >= 3) & (df['ipl_team_score'] >= 2800)).astype(int)

        print("Processed DataFrame:")
        print(df.head())

        # Line 8: Convert the processed DataFrame back to CSV format (without index)
        processed_csv_content = df.to_csv(index=False)

        # Line 9: Upload the processed CSV back to S3
        s3_client.put_object(Bucket=bucket_name, Key=output_file_key, Body=processed_csv_content)

        print(f"Successfully processed {input_file_key} and saved to {output_file_key} in {bucket_name}")

        return {
            'statusCode': 200,
            'body': json.dumps(f'Successfully processed data and stored in s3://{bucket_name}/{output_file_key}')
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error processing data: {str(e)}')
        }

# Python Code Explanation (No Pandas) for AWS Lambda

This document provides a line-by-line explanation of the provided Python code, which processes CSV data without relying on the Pandas library, formatted for easy viewing and understanding in a Google Colab notebook.

### `import csv, StringIO`

import csv, StringIO

-   **Explanation:** This line imports the necessary Python modules.
    
    -   `csv`: This module provides classes and functions to facilitate the reading and writing of tabular data in CSV format.
        
    -   `StringIO`: This module allows you to treat a string as an in-memory text file. This is crucial for `csv.reader` to process string content as if it were reading from a physical file.
        

### `csv_file = StringIO(csv_content)`

csv_file = StringIO(csv_content)

-   **Explanation:** This line takes the raw CSV content (assumed to be a string variable named `csv_content`, likely obtained from an S3 object) and wraps it within a `StringIO` object. This makes the string behave like a file, which is a prerequisite for the `csv.reader` to be able to parse it.
    

### `reader = csv.reader(csv_file)`

reader = csv.reader(csv_file)

-   **Explanation:** This creates a `reader` object from the `csv` module. This `reader` object is an iterator that will iterate over lines in the `csv_file` (our `StringIO` object) and parse them into lists of strings, handling CSV specific formatting like delimiters and quotes.
    

### `header = next(reader)`

header = next(reader)

-   **Explanation:** The `next()` function is used to retrieve the first item from an iterator. In this context, it reads the very first row of the CSV file, which is typically the header row containing column names. This row is then stored in the `header` list.
    

### `data = list(reader)`

data = list(reader)

-   **Explanation:** After the `header` has been read, the `reader` iterator is positioned at the second row. Calling `list(reader)` consumes the rest of the iterator, reading all remaining rows from the CSV file. Each row is parsed into a list of strings, and all these row lists are collected into a single list called `data`.
    

### `header.append('can_win')`

header.append('can_win')

-   **Explanation:** This line modifies the `header` list by adding a new string, `'can_win'`, to its end. This prepares the header for the new column that will be generated based on the processing logic.
    

### `for row in data:`

for row in data:

-   **Explanation:** This initiates a loop that iterates through each `row` in the `data` list. Each `row` inside this loop will be a list of strings representing a single record from the original CSV file (excluding the header).
    

### `ipl_wins = int(row[1]), ipl_team_score = int(row[2])`

ipl_wins = int(row[1]) ipl_team_score = int(row[2])

-   **Explanation:** Inside the loop, these lines extract specific values from the current `row`.
    
    -   `row[1]`: Accesses the element at index 1 (the second column) of the current row, which is assumed to be the 'ipl_wins' value.
        
    -   `row[2]`: Accesses the element at index 2 (the third column) of the current row, assumed to be the 'ipl_team_score' value.
        
    -   `int(...)`: Converts these string values into integers, as numerical comparisons are needed for the classification logic.
        

### `can_win = 1 if (ipl_wins >= 3 and ipl_team_score >= 2800) else 0`

can_win = 1 if (ipl_wins >= 3 and ipl_team_score >= 2800) else 0

-   **Explanation:** This is the core classification logic, implemented using a conditional expression (ternary operator).
    
    -   `ipl_wins >= 3 and ipl_team_score >= 2800`: This checks if both conditions are true: the team has 3 or more IPL wins AND an IPL team score of 2800 or more.
        
    -   `1 if ... else 0`: If both conditions are `True`, `can_win` is assigned the integer `1`. Otherwise, it's assigned `0`.
        

### `new_row = row + [str(can_win)]`

new_row = row + [str(can_win)]

-   **Explanation:** This line constructs a `new_row` for the processed data.
    
    -   `row`: Takes the original list of string values for the current row.
        
    -   `[str(can_win)]`: Creates a new list containing only the `can_win` value, which is converted back to a string because all elements in a CSV row are typically strings.
        
    -   `+`: Concatenates the original `row` list with the new list containing `can_win`, effectively appending the `can_win` value as a new column to the row.
        

### `processed_csv_content = output_csv_file.getvalue()`

processed_csv_content = output_csv_file.getvalue()

-   **Explanation:** (Assuming `output_csv_file` is a `StringIO` object where processed rows are being written by a `csv.writer`) This line retrieves the complete string content that has been written to the `output_csv_file` (the `StringIO` object). This string will contain the header followed by all the processed data rows, formatted as a CSV.
    

### `The rest is similar to the Pandas version for S3 interaction and error handling.`

The rest is similar to the Pandas version for S3 interaction and error handling.

-   **Explanation:** This statement indicates that the subsequent parts of the Lambda function (uploading the `processed_csv_content` back to S3 using `s3_client.put_object` and the `try-except` block for error handling) would be identical in structure and function to the Pandas version previously explained.
    

### Common AWS Error: "Invalid Lambda function policy."

Common AWS Error: "Invalid Lambda function policy." This might happen if you are trying to attach an invalid policy or have a typo. Review the policy syntax.

-   **Explanation:** This is a common error encountered when deploying AWS Lambda functions. It suggests that the IAM (Identity and Access Management) policy attached to your Lambda function's execution role is malformed, contains incorrect syntax, or grants permissions in an invalid way. To resolve this, you would need to carefully review the JSON policy document for syntax errors, ensure all actions and resources are correctly specified, and verify that the policy adheres to IAM policy best practices.

In [ ]:
import json
import boto3
import os
import csv
from io import StringIO

s3_client = boto3.client('s3')

def lambda_handler(event, context):
    bucket_name = os.environ.get('S3_BUCKET_NAME')
    input_file_key = 'ipl_data.csv'
    output_file_key = 'processed_ipl_data.csv'

    print(f"Reading {input_file_key} from bucket {bucket_name}")

    try:
        response = s3_client.get_object(Bucket=bucket_name, Key=input_file_key)
        csv_content = response['Body'].read().decode('utf-8')

        # Use StringIO to treat the string as a file for csv.reader
        csv_file = StringIO(csv_content)
        reader = csv.reader(csv_file)
        header = next(reader) # Read header row
        data = list(reader)  # Read remaining data rows

        # Add 'can_win' to header
        header.append('can_win')

        processed_rows = []
        for row in data:
            ipl_wins = int(row[1]) # ipl_wins is the second column (index 1)
            ipl_team_score = int(row[2]) # ipl_team_score is the third column (index 2)

            # Simple preprocessing logic
            can_win = 1 if (ipl_wins >= 3 and ipl_team_score >= 2800) else 0
            new_row = row + [str(can_win)] # Append the new value
            processed_rows.append(new_row)

        # Prepare content for writing back to S3
        output_csv_file = StringIO()
        writer = csv.writer(output_csv_file)
        writer.writerow(header) # Write header
        writer.writerows(processed_rows) # Write processed rows
        processed_csv_content = output_csv_file.getvalue()

        s3_client.put_object(Bucket=bucket_name, Key=output_file_key, Body=processed_csv_content)

        print(f"Successfully processed {input_file_key} and saved to {output_file_key} in {bucket_name}")

        return {
            'statusCode': 200,
            'body': json.dumps(f'Successfully processed data and stored in s3://{bucket_name}/{output_file_key}')
        }
    except Exception as e:
        print(f"Error processing data: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error processing data: {str(e)}')
        }

In [ ]:
# Line 1: Install scikit-learn if it's not already available in the environment
!pip install scikit-learn
!pip install pandas

import pandas as pd
import boto3
import io
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
import joblib # For saving/loading models

# Line 1: Define your S3 bucket name
bucket_name = 'ipl-mlops-workshop-yourname-date' # REPLACE with your S3 bucket name
# Line 2: Define the key for the processed data file
processed_data_key = 'processed_ipl_data.csv'
# Line 3: Define the key for saving the trained model
model_key = 'ipl_winner_predictor.joblib'

# Line 4: Initialize S3 client
s3 = boto3.client('s3')

print(f"Downloading {processed_data_key} from {bucket_name}...")
# Line 5: Get the object from S3
obj = s3.get_object(Bucket=bucket_name, Key=processed_data_key)
# Line 6: Read the object body and decode it
body = obj['Body'].read().decode('utf-8')
# Line 7: Use io.StringIO to read the string content as a CSV file
df = pd.read_csv(io.StringIO(body))

print("Data loaded successfully:")
print(df.head())
print(df.info())

# Line 1: Define features (X) and target (y)
X = df[['ipl_wins', 'ipl_team_score']] # Features for prediction
y = df['can_win'] # Target variable

print("Features (X) head:")
print(X.head())
print("Target (y) head:")
print(y.head())

# Line 2: Split data into training and testing sets (80% train, 20% test)
# random_state for reproducibility
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

# Line 1: Initialize the Logistic Regression model
model = LogisticRegression(random_state=42)

print("Training model...")
# Line 2: Train the model using the training data
model.fit(X_train, y_train)

print("Model training complete.")

# Line 3: Evaluate the model on the test set
accuracy = model.score(X_test, y_test)
print(f"Model accuracy on test set: {accuracy:.2f}")

# Line 1: Save the trained model locally as a joblib file
local_model_path = '/tmp/ipl_winner_predictor.joblib' # SageMaker notebook instances have /tmp for temporary storage
joblib.dump(model, local_model_path)

print(f"Model saved locally to {local_model_path}")

# Line 2: Upload the saved model to S3
s3.upload_file(local_model_path, bucket_name, model_key)

print(f"Model uploaded to s3://{bucket_name}/{model_key}")

ModuleNotFoundError: No module named 'boto3'

In [ ]:
import json
import boto3
import os
import joblib
import io
import pandas as pd # Although we avoided for preprocessing, it's common for inference

# Line 1: Initialize S3 client
s3_client = boto3.client('s3')

# Line 2: Define S3 bucket and model key from environment variables
BUCKET_NAME = os.environ.get('S3_BUCKET_NAME')
MODEL_KEY = 'ipl_winner_predictor.joblib'
LOCAL_MODEL_PATH = '/tmp/ipl_winner_predictor.joblib' # Temporary path in Lambda execution environment

# Line 3: Global variable to store the loaded model, to avoid re-loading on subsequent invocations (warm start)
model = None

def load_model():
    global model
    if model is None:
        print(f"Loading model from s3://{BUCKET_NAME}/{MODEL_KEY}...")
        try:
            # Line 4: Download model from S3
            s3_client.download_file(BUCKET_NAME, MODEL_KEY, LOCAL_MODEL_PATH)
            # Line 5: Load the model using joblib
            model = joblib.load(LOCAL_MODEL_PATH)
            print("Model loaded successfully.")
        except Exception as e:
            print(f"Error loading model: {e}")
            raise e
    return model

def lambda_handler(event, context):
    # Line 6: Load the model (or use existing loaded model)
    current_model = load_model()

    # Line 7: Parse the request body (assuming JSON input from API Gateway)
    try:
        body = json.loads(event['body'])
        ipl_wins = body['ipl_wins']
        ipl_team_score = body['ipl_team_score']
    except KeyError:
        return {
            'statusCode': 400,
            'body': json.dumps('Missing "ipl_wins" or "ipl_team_score" in request body.')
        }
    except json.JSONDecodeError:
        return {
            'statusCode': 400,
            'body': json.dumps('Invalid JSON in request body.')
        }
    except Exception as e:
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error parsing input: {str(e)}')
        }

    # Line 8: Prepare input for the model (must match training features order)
    # Create a DataFrame for consistent input to scikit-learn model
    input_data = pd.DataFrame([[ipl_wins, ipl_team_score]], columns=['ipl_wins', 'ipl_team_score'])

    # Line 9: Make prediction
    try:
        prediction = current_model.predict(input_data)[0]
        # Line 10: Convert numpy.int64 to standard int for JSON serialization
        prediction_result = int(prediction)
        print(f"Prediction for ipl_wins={ipl_wins}, ipl_team_score={ipl_team_score}: {prediction_result}")

        return {
            'statusCode': 200,
            'headers': {
                'Content-Type': 'application/json'
            },
            'body': json.dumps({
                'prediction': prediction_result,
                'message': 'Team can win' if prediction_result == 1 else 'Team cannot win'
            })
        }
    except Exception as e:
        print(f"Error during prediction: {e}")
        return {
            'statusCode': 500,
            'body': json.dumps(f'Error during prediction: {str(e)}')
        }

In [ ]:
# ✅ Step 1: Clean and prepare directories
!rm -rf python joblib_pandas_sklearn_layer.zip
!mkdir -p python

# ✅ Step 2: Install required packages into 'python/' (Lambda layer structure)
!pip install joblib -t python/

# ✅ Step 3: Zip it into a Lambda-compatible layer
!zip -r joblib_pandas_sklearn_layer.zip python > /dev/null

# ✅ Step 4: Download the zip file
from google.colab import files
files.download("joblib_pandas_sklearn_layer.zip")


  Using cached pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (91 kB)
  Using cached joblib-1.5.1-py3-none-any.whl.metadata (5.6 kB)
  Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (62 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached pytz-2025.2-py2.py3-none-any.whl.metadata (22 kB)
  Using cached tzdata-2025.2-py2.py3-none-any.whl.metadata (1.4 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached pandas-2.3.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (12.4 MB)
Using cached joblib-1.5.1-py3-none-any.whl (307 kB)
Using cached numpy-2.3.1-cp311-cp311-manylinux_2_28_x86_64.whl (16.9 MB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached pytz-2025.2-py2.py3-none-any.whl (509 kB)
Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
ERROR: pip's de

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>